In [1]:
import os

In [2]:
import pickle

In [3]:
import subprocess, select

In [4]:
from numpy.random import rand

In [5]:
engine_str = """\
import pickle
import threading
import sys

 
sys.stdout._line_buffering = True


def writer(str_lst):
    pickle.dump(str_lst, sys.stdout.buffer)
    

if __name__ == "__main__":
    str_lst = []
    while True:
        input_str = pickle.load(sys.stdin.buffer)
        with open('progress.txt', 'a') as f:
            f.write(str(input_str))
        if input_str == "close":
            break
        elif input_str == "reply": 
            thread = threading.Thread(target=writer, args=[str_lst])
            thread.start()
            thread.join()
            # pickle.dump(str_lst, sys.stdout.buffer)
            sys.stdout.flush()
            with open('final.txt', 'ab') as f:
                f.write(pickle.dumps(str_lst))
        else: 
            str_lst.append(input_str)
"""

In [6]:
if not os.path.exists('engine.py'):
    with open('engine.py', 'w') as f:
        f.writelines(engine_str)

In [7]:
proc = subprocess.Popen(
    'python engine.py', 
    shell=True, 
    stdin=subprocess.PIPE,                      
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    bufsize=8192
)

In [8]:
lst = []
for i in range(1000):
    pickle.dump('%d\n' % i, proc.stdin)
    lst.append('%d\n' % i)
proc.stdin.flush()

In [9]:
pickle.dump('reply', proc.stdin)

In [10]:
proc.stdin.flush()

In [11]:
data = b""
i = 0
count = 0
while True:
    
    print('before poll')
    # proc.wait()
    proc.poll()
    print('after poll')
    ready = select.select([proc.stdout, proc.stderr], [], [], 1.0)
    print('after select', proc.returncode)
    if proc.stdout in ready[0] and count == 0:
        print('after if')
        # packet = proc.stdout.read(8192)
        packet = proc.stdout.read(1024)
        # packet = proc.stdout.read()
        print('packet: ', packet , packet == b'', packet == ' ')
        print('after read')
        if len(packet) == 0: 
            break
        else:
            data += packet
        print(str(i))
        i += 1
        count = 0
        # count += 1
    elif count > 2:
        break
    else:
        count += 1
        print('not ready')

before poll
after poll
after select None
after if
packet:  b'\x80\x03]q\x00(X\x02\x00\x00\x000\nq\x01X\x02\x00\x00\x001\nq\x02X\x02\x00\x00\x002\nq\x03X\x02\x00\x00\x003\nq\x04X\x02\x00\x00\x004\nq\x05X\x02\x00\x00\x005\nq\x06X\x02\x00\x00\x006\nq\x07X\x02\x00\x00\x007\nq\x08X\x02\x00\x00\x008\nq\tX\x02\x00\x00\x009\nq\nX\x03\x00\x00\x0010\nq\x0bX\x03\x00\x00\x0011\nq\x0cX\x03\x00\x00\x0012\nq\rX\x03\x00\x00\x0013\nq\x0eX\x03\x00\x00\x0014\nq\x0fX\x03\x00\x00\x0015\nq\x10X\x03\x00\x00\x0016\nq\x11X\x03\x00\x00\x0017\nq\x12X\x03\x00\x00\x0018\nq\x13X\x03\x00\x00\x0019\nq\x14X\x03\x00\x00\x0020\nq\x15X\x03\x00\x00\x0021\nq\x16X\x03\x00\x00\x0022\nq\x17X\x03\x00\x00\x0023\nq\x18X\x03\x00\x00\x0024\nq\x19X\x03\x00\x00\x0025\nq\x1aX\x03\x00\x00\x0026\nq\x1bX\x03\x00\x00\x0027\nq\x1cX\x03\x00\x00\x0028\nq\x1dX\x03\x00\x00\x0029\nq\x1eX\x03\x00\x00\x0030\nq\x1fX\x03\x00\x00\x0031\nq X\x03\x00\x00\x0032\nq!X\x03\x00\x00\x0033\nq"X\x03\x00\x00\x0034\nq#X\x03\x00\x00\x0035\nq$X\x03\x00\x00\x0036

after select None
not ready
before poll
after poll
after select None
not ready
before poll
after poll
after select None
not ready
before poll
after poll
after select None


In [12]:
with open('final.txt', 'rb') as f:
    data_file = f.read()

In [13]:
len(data), len(data_file)

(9216, 13133)

In [14]:
data[-100:-1], data_file[-100:-1]

(b'\x00\x00713\nr\xca\x02\x00\x00X\x04\x00\x00\x00714\nr\xcb\x02\x00\x00X\x04\x00\x00\x00715\nr\xcc\x02\x00\x00X\x04\x00\x00\x00716\nr\xcd\x02\x00\x00X\x04\x00\x00\x00717\nr\xce\x02\x00\x00X\x04\x00\x00\x00718\nr\xcf\x02\x00\x00X\x04\x00\x00\x00719\nr\xd0\x02\x00\x00X\x04\x00\x00',
 b'X\x04\x00\x00\x00993\nr\xe2\x03\x00\x00X\x04\x00\x00\x00994\nr\xe3\x03\x00\x00X\x04\x00\x00\x00995\nr\xe4\x03\x00\x00X\x04\x00\x00\x00996\nr\xe5\x03\x00\x00X\x04\x00\x00\x00997\nr\xe6\x03\x00\x00X\x04\x00\x00\x00998\nr\xe7\x03\x00\x00X\x04\x00\x00\x00999\nr\xe8\x03\x00\x00e')